In [1]:
%cd /home

/home


In [7]:
# Setting up all directory
root_folder = "/home/"

data_directory = root_folder+"data/raw/"
new_data_directory = root_folder+"data/new/"
data_profile_path = root_folder+"/data/profile_report/"
intermediate_data_path = root_folder+"data/interim/"
print("directory loaded")

directory loaded


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
%matplotlib inline
from scripts import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from pandas_profiling import ProfileReport

In [9]:
%%time
members_n, user_logs_n, transactions_n, train_n  = utils.load_data( [
                                                            f"{new_data_directory}members_profile_new.csv",
                                                            f"{new_data_directory}user_logs_new.csv",
                                                            f"{new_data_directory}transactions_logs_new.csv",
                                                            f"{new_data_directory}churn_logs_new.csv"
                                                            ]
                                                          )

CPU times: user 25.1 s, sys: 3.29 s, total: 28.4 s
Wall time: 28.4 s


In [16]:
print(members_n.shape)
print(transactions_n.shape)
print(user_logs_n.shape)
print(train_n.shape)

(6769473, 6)
(1431009, 9)
(18396362, 9)
(970960, 2)


In [12]:
user_logs_n['date'] = utils.fix_time_in_df(user_logs_n, 'date', expand=False)

In [30]:
%%time
members, user_logs, transactions, train  = utils.load_data( [
                                                            f"{data_directory}members_profile.csv",
                                                            f"{data_directory}userlogs.csv",
                                                            f"{data_directory}transactions_logs.csv",
                                                            f"{data_directory}churn_logs.csv"
                                                            ]
                                                          )

CPU times: user 10.9 s, sys: 1.7 s, total: 12.6 s
Wall time: 12.6 s


In [15]:
print(members.shape)
print(transactions.shape)
print(user_logs.shape)
print(train.shape)

(4348970, 6)
(4380726, 9)
(4828886, 9)
(385591, 2)


In [13]:
user_logs_n.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,2017-03-31,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,2017-03-30,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,2017-03-31,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,2017-03-31,2,1,0,1,112,93,28401.558


In [31]:
members_list = np.unique(list(members['msno']))

In [34]:
train_members_list = np.unique(list(train['msno']))

In [35]:
user_logs_n[(user_logs_n['date']>'2017-03-01') & 
            (user_logs_n['date']<'2017-03-31') &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))].shape

(5014919, 9)

In [38]:
user_logs_n[(user_logs_n['date']>'2017-03-01') & 
            (user_logs_n['date']<'2017-03-31') &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))].to_csv("/home/data/new_append/user_logs_march.csv",index=False)

In [22]:
%%time
transactions_n['transaction_date'] = utils.fix_time_in_df(transactions_n, 'transaction_date', expand=False)
transactions_n['membership_expire_date'] = utils.fix_time_in_df(transactions_n, 'membership_expire_date', expand=False)

CPU times: user 1.39 s, sys: 116 ms, total: 1.5 s
Wall time: 1.5 s


In [23]:
transactions_n.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0


In [26]:
transactions_n[(transactions_n['transaction_date']>'2017-03-01') & (transactions_n['transaction_date']<'2017-03-31') & (transactions_n['membership_expire_date']<'2017-12-31')].to_csv("/home/data/new_append/transactions_logs_march.csv",index=False)

In [37]:
transactions_n[(transactions_n['transaction_date']>'2017-03-01') & 
               (transactions_n['transaction_date']<'2017-03-31') & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))].shape

(307092, 9)

In [39]:
transactions_n[(transactions_n['transaction_date']>'2017-03-01') & 
               (transactions_n['transaction_date']<'2017-03-31') & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))].to_csv("/home/data/new_append/transactions_logs_march.csv",index=False)

In [40]:
march_user_logs = user_logs_n[(user_logs_n['date']>'2017-03-01') & 
            (user_logs_n['date']<'2017-03-31') &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))]

In [41]:
march_transactions = transactions_n[(transactions_n['transaction_date']>'2017-03-01') & 
               (transactions_n['transaction_date']<'2017-03-31') & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))]

In [42]:
user_logs_updated = user_logs.append(march_user_logs)

In [64]:
user_logs_updated.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,kvn0x25i/D06AX1K3Sv9djeZA5oRsjPm8ysAl4rzBYs=,20151029,8,1,1,2,54,58,13573.105
1,kvn0x25i/D06AX1K3Sv9djeZA5oRsjPm8ysAl4rzBYs=,20160103,0,0,0,1,27,28,6233.864
2,kvn0x25i/D06AX1K3Sv9djeZA5oRsjPm8ysAl4rzBYs=,20160302,19,5,2,0,69,78,17517.728
3,kvn0x25i/D06AX1K3Sv9djeZA5oRsjPm8ysAl4rzBYs=,20160428,26,0,0,0,1,27,278.301
4,kvn0x25i/D06AX1K3Sv9djeZA5oRsjPm8ysAl4rzBYs=,20160628,2,0,0,0,0,2,11.345


In [43]:
user_logs_updated.shape

(9843805, 9)

In [44]:
transactions_updated = transactions.append(march_transactions)
transactions_updated.shape

(4687818, 9)

In [45]:
def get_new_data_appended(new_data_directory, start_data, end_date):
    members_n, user_logs_n, transactions_n, train_n  = utils.load_data( [
                                                            f"{new_data_directory}members_profile_new.csv",
                                                            f"{new_data_directory}user_logs_new.csv",
                                                            f"{new_data_directory}transactions_logs_new.csv",
                                                            f"{new_data_directory}churn_logs_new.csv"
                                                            ]
                                                          )
    
    #get the list of memebers fron historical data. This assumes, no new user has been added in the system. Shouldn't be done, when new users are added
    members_list = np.unique(list(members['msno']))
    train_members_list = np.unique(list(train['msno']))
    
    #Some Date Filters are manual at this point for sanity check 
    user_logs_n['date'] = utils.fix_time_in_df(user_logs_n, 'date', expand=False)
    march_user_logs = user_logs_n[(user_logs_n['date']>start_data) & 
            (user_logs_n['date']<end_date) &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))]
    
    transactions_n['transaction_date'] = utils.fix_time_in_df(transactions_n, 'transaction_date', expand=False)
    transactions_n['membership_expire_date'] = utils.fix_time_in_df(transactions_n, 'membership_expire_date', expand=False)
    march_transactions = transactions_n[(transactions_n['transaction_date']>start_data) & 
               (transactions_n['transaction_date']<end_date) & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))]

    
    return march_user_logs, march_transactions

In [46]:
%%time
start_data = '2017-03-01'
end_date = '2017-03-31'
new_data_directory = root_folder+"data/new/"

a,b = get_new_data_appended(new_data_directory, start_data, end_date)

CPU times: user 50.3 s, sys: 6.61 s, total: 56.9 s
Wall time: 56.8 s


In [49]:
members_n.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [90]:
def get_change(current, previous):
    if current == previous:
        return 0
    try:
        return (abs(current - previous) / previous) * 100.0
    except ZeroDivisionError:
        return float('inf')
    
def get_data_drift(current_data, old_data, column_list,exclude_list, metric='std'):
    std_deviation_percentage = []
    mean_deviation_percentage = []
    for eachCol in column_list:
        if metric == 'std' and eachCol not in exclude_list:
            std1 = current_data[eachCol].std()
            std2 = old_data[eachCol].std()
            std_deviation_percentage.append(get_change(std1, std2))
        elif metric =='mean'and eachCol not in exclude_list:
            mean1 = current_data[eachCol].mean()
            mean2 = old_data[eachCol].mean()
            mean_deviation_percentage.append(get_change(mean1, mean2))
    print(std_deviation_percentage,mean_deviation_percentage)
    return np.mean(std_deviation_percentage), np.mean(mean_deviation_percentage)

In [95]:
column_list = list(user_logs.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(user_logs_n, user_logs, column_list,exclude_list= ['date'], metric='std')

['date', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs']
[8.202758342221658, 10.71451630822849, 10.481820946531997, 42.140590737945686, 12.405514033808252, 1.1186882422743198, 99.99999998437839] []


(26.437698370769827, nan)

In [96]:
column_list = list(user_logs.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(user_logs_n, user_logs, column_list,exclude_list= ['date'], metric='mean')

['date', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs']
[] [10.660760937669314, 10.817536404550486, 9.302931212476622, 2.403962858140031, 7.075398563404825, 0.934810590264208, -100.00000174622627]


(nan, -8.400657311388684)

In [93]:
column_list = list(transactions.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(transactions_n, transactions, column_list,exclude_list= ['transaction_date','membership_expire_date'], metric='std')

['payment_method_id', 'payment_plan_days', 'plan_list_price', 'actual_amount_paid', 'is_auto_renew', 'transaction_date', 'membership_expire_date', 'is_cancel']
[77.99252836046986, 667.4599089415049, 586.4353306148355, 580.058025422085, 33.40287126038689, 35.28128439902486] []


(330.10499149971787, nan)

In [97]:
column_list = list(transactions.select_dtypes(include=['int','float']).columns)
print(column_list)
get_data_drift(transactions_n, transactions, column_list,exclude_list= ['transaction_date','membership_expire_date'], metric='mean')

['payment_method_id', 'payment_plan_days', 'plan_list_price', 'actual_amount_paid', 'is_auto_renew', 'transaction_date', 'membership_expire_date', 'is_cancel']
[] [4.900430373078422, 115.99443117764416, 129.9453706176852, 130.33607214722778, 12.161534775048644, 85.12826427699356]


(nan, 79.74435056127963)

In [ ]:
def get_new_data_appended(new_data_directory, start_data, end_date):
    members_n, user_logs_n, transactions_n, train_n  = utils.load_data( [
                                                            f"{new_data_directory}members_profile_new.csv",
                                                            f"{new_data_directory}user_logs_new.csv",
                                                            f"{new_data_directory}transactions_logs_new.csv",
                                                            f"{new_data_directory}churn_logs_new.csv"
                                                            ]
                                                          )
    
    #get the list of memebers fron historical data. This assumes, no new user has been added in the system. Shouldn't be done, when new users are added
    members_list = np.unique(list(members['msno']))
    train_members_list = np.unique(list(train['msno']))
    
    #Some Date Filters are manual at this point for sanity check 
    user_logs_n['date'] = utils.fix_time_in_df(user_logs_n, 'date', expand=False)
    march_user_logs = user_logs_n[(user_logs_n['date']>start_data) & 
            (user_logs_n['date']<end_date) &
            (user_logs_n['msno'].isin(members_list)) & 
            (user_logs_n['msno'].isin(train_members_list))]
    
    transactions_n['transaction_date'] = utils.fix_time_in_df(transactions_n, 'transaction_date', expand=False)
    transactions_n['membership_expire_date'] = utils.fix_time_in_df(transactions_n, 'membership_expire_date', expand=False)
    march_transactions = transactions_n[(transactions_n['transaction_date']>start_data) & 
               (transactions_n['transaction_date']<end_date) & 
               (transactions_n['membership_expire_date']<'2017-12-31') & 
               (transactions_n['msno'].isin(members_list)) & 
               (transactions_n['msno'].isin(train_members_list))]

    
    return march_user_logs, march_transactions